In [1]:
!pip install langchain langchain-community langchain-experimental langchain-google-genai neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.45
    Uninstalling langchain-core-0.3.45:
      Successfully uninstalled langchain-core-0.3.45
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully un

In [1]:
import os
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.documents import Document

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key="",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
llmtransformer = LLMGraphTransformer(llm=llm)

In [3]:
llm_add = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
    google_api_key="",
    temperature=0,
    allow_dangerous_requests=True,
    system_message="""You are a specialized assistant for generating Neo4j Cypher queries to ADD or UPDATE medical data in a database.

The database contains the following entity types:
- Body_part, Brain_region, Device, Disease, Medical_condition, Medical_finding, Medical_procedure
- Medical_record, Medication, Metric, Nutrition, Patient, Person, Place, Procedure, Symptom

And relationships between entities such as:
- ADMITTED_TO, ASSISTED, ASSISTED_IN, CHILD_OF, CONNECTED_TO, DIAGNOSED_WITH
- DONOR_FOR, FED_WITH, FOLLOWING, HAS_CONDITION, HAS_FINDING, HAS_HISTORY_OF
- HAS_MEDICAL_HISTORY, HAS_PROCEDURE, HAS_SURGICAL_HISTORY, HAS_SYMPTOM
- HOSPITAL_COMPLICATION, HOSPITAL_DIAGNOSIS, LIKELY, LOCATION_OF, PERFORMED
- POSSIBLE_FINDING, REASON_FOR_ADMISSION, RECEIVED, RECEIVED_PROCEDURE
- SECONDARY_TO, SUSPECTED_DIAGNOSIS, TRANSFERRED_FROM, TRANSFERRED_TO, UNDERWENT

When a user provides information, generate Cypher queries to:
1. CREATE new nodes with appropriate labels and properties
2. MERGE existing nodes when appropriate to avoid duplicates
3. CREATE relationships between nodes with correct relationship types
4. UPDATE properties of existing nodes when new information is provided

Always generate complete, executable Cypher queries with proper syntax for adding data to the Neo4j database.
"""
)
llmtransformer = LLMGraphTransformer(llm=llm_add)

In [42]:
llm_query = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
    google_api_key="",
    temperature=0,
    allow_dangerous_requests=True,
    system_message="""use below query to get the response from the graph database no need to specify name or anything just use it as is
        MATCH (source)-[relationship]->(target)
        RETURN source.name as Source, type(relationship) as Relationship, target.name as Target""",
)


In [4]:
graph = Neo4jGraph()

<ipython-input-4-6e7385464d00>:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [31]:
text = """    mr. is a 72-year-old male with a past medical history significant for pancreatic cancer, ulcerative colitis, hypertension, status post endoscopic retrograde cholangiopancreatography, and status post total abdominal colectomy 20 years ago with an end-ileostomy. the patient underwent an endoscopic retrograde cholangiopancreatography recently, but a stent was unable to be placed. a computed tomography was performed which demonstrated a head of the pancreas mass with dilated intrahepatic duct along with vascular involvement of the gastroduodenal artery and superior mesenteric vein. he presented for exploratory laparotomy with possible pancreatic mass resection. past medical history: 1. ulcerative colitis. 2. hypertension. 3. benign prostatic hypertrophy. past surgical history: 1. total abdominal colectomy with end-ileostomy. 2. status post transurethral resection of prostate. medications on admission: 1. moexipril 15 mg by mouth once per day. 2. aspirin 81 mg by mouth once per day. 3. atenolol 25 mg by mouth once per day. 4. allopurinol 300 mg by mouth once per day. 5. multivitamin. allergies: the patient has no known drug allergies. physical examination on presentation: the patient is a thin, cachectic caucasian male who was alert and oriented times three. in no apparent distress. the sclerae were anicteric. the patient was jaundiced. the oropharynx was clear with moist mucous membranes. the neck was supple and without lymphadenopathy. the heart was regular in rate and rhythm. the lungs were clear to auscultation bilaterally. the abdomen was soft, nontender, and nondistended. there was a well-healed midline scar and ileostomy present. the extremities were warm without cyanosis, clubbing, or edema. pertinent laboratory values on presentation: his hematocrit was 43.2. his inr was 1.2. creatinine was 1.6. aspartate aminotransferase was 51, his alanine-aminotransferase was 89, his alkaline phosphatase was 395, and his total bilirubin was 12.5. brief summary of hospital course: on the day of admission, the patient was taken to the operating room where an exploratory laparotomy was performed. the patient had evidence of unresectable pancreatic cancer with biliary obstruction seen intraoperatively. adhesiolysis was therefore performed along with a roux-en-y hepaticojejunostomy, and open cholecystectomy, an open pancreatic biopsy, and a gastrojejunostomy. the estimated blood loss for the procedure was 250 cc. the patient was discharged to the regular hospital floor after being extubated in the postanesthesia care unit in good condition. in the evening on postoperative day one, the patient was taken back to the operating room emergently for likely mesenteric bleeding. this was controlled with suture ligation, and the patient was admitted to the surgical intensive care unit postoperatively for close monitoring. the patient remained intubated in the intensive care unit on pressor support and received total parenteral nutrition until postoperative day seven. at this time, the patient's mental status was extremely labile requiring haldol for agitation. the patient's hematocrit was stable at 35.8 at this time. tube feeds were initiated on postoperative day eight. on postoperative day nine, the patient was transferred to the regular hospital floor. at this time, tube feeds were held for elevated residuals and nausea. he was still receiving total parenteral nutrition at this time. the patient's mental status was still not completely improved. a computed tomography scan was performed on postoperative day ten which did not demonstrate any intra-abdominal pathology. the patient was started on sips on postoperative day eleven and was started on his home medications. at this time, he was seen by the physical therapy service and was being screened for rehabilitation placement. however, on the evening on postoperative day twelve the patient spiked a temperature to 101.5 degrees fahrenheit. a fever workup was done including a chest x-ray and blood cultures. early the next morning, the patient was found unresponsive without a pulse at approximately 2:45 a.m. at this time, a code blue was called and advanced cardiac life support protocol was initiated. however, the patient was asystolic without any respiratory effort at this time. he did receive multiple rounds of epinephrine along with attempts at ventilation. however, the patient never regained electrical activity and was pronounced deceased at 2:57 a.m. the patient's wife was notified at this time. however, a postmortem examination was declined. condition at discharge: the patient expired on . , m.d. dictated by: medquist36 Procedure: Continuous invasive mechanical ventilation for less than 96 consecutive hours Venous catheterization, not elsewhere classified Parenteral infusion of concentrated nutritional substances Insertion of endotracheal tube Other lysis of peritoneal adhesions Reopening of recent laparotomy site Cardiopulmonary resuscitation, not otherwise specified Incisional hernia repair Anastomosis of hepatic duct to gastrointestinal tract Open biopsy of pancreas Diagnoses: Unspecified essential hypertension Cardiac complications, not elsewhere classified Hemorrhage complicating a procedure Cardiac arrest Malignant neoplasm of head of pancreas Calculus of gallbladder with other cholecystitis, without mention of obstruction Peritoneal adhesions (postoperative) (postinfection) Incisional hernia without mention of obstruction or gangrene Surgical operation with anastomosis, bypass, or graft, with natural or artificial tissues used as implant causing abnormal patient reaction, or later complication, without mention of misadventure at time of operation

 """

In [32]:
documents=[Document(page_content=text)]
graph_doc = llmtransformer.convert_to_graph_documents(documents)

In [33]:
graph_doc

[GraphDocument(nodes=[Node(id='Mr.', type='Person', properties={}), Node(id='Pancreatic Cancer', type='Disease', properties={}), Node(id='Ulcerative Colitis', type='Disease', properties={}), Node(id='Hypertension', type='Disease', properties={}), Node(id='Endoscopic Retrograde Cholangiopancreatography', type='Medical procedure', properties={}), Node(id='Total Abdominal Colectomy', type='Medical procedure', properties={}), Node(id='End-Ileostomy', type='Medical procedure', properties={}), Node(id='Computed Tomography', type='Medical procedure', properties={}), Node(id='Head Of The Pancreas Mass', type='Medical condition', properties={}), Node(id='Dilated Intrahepatic Duct', type='Medical condition', properties={}), Node(id='Vascular Involvement', type='Medical condition', properties={}), Node(id='Gastroduodenal Artery', type='Anatomy', properties={}), Node(id='Superior Mesenteric Vein', type='Anatomy', properties={}), Node(id='Exploratory Laparotomy', type='Medical procedure', propertie

In [34]:
for node in graph_doc[0].nodes:
    print(node)

id='Mr.' type='Person' properties={}
id='Pancreatic Cancer' type='Disease' properties={}
id='Ulcerative Colitis' type='Disease' properties={}
id='Hypertension' type='Disease' properties={}
id='Endoscopic Retrograde Cholangiopancreatography' type='Medical procedure' properties={}
id='Total Abdominal Colectomy' type='Medical procedure' properties={}
id='End-Ileostomy' type='Medical procedure' properties={}
id='Computed Tomography' type='Medical procedure' properties={}
id='Head Of The Pancreas Mass' type='Medical condition' properties={}
id='Dilated Intrahepatic Duct' type='Medical condition' properties={}
id='Vascular Involvement' type='Medical condition' properties={}
id='Gastroduodenal Artery' type='Anatomy' properties={}
id='Superior Mesenteric Vein' type='Anatomy' properties={}
id='Exploratory Laparotomy' type='Medical procedure' properties={}
id='Pancreatic Mass Resection' type='Medical procedure' properties={}
id='Benign Prostatic Hypertrophy' type='Disease' properties={}
id='Tran

In [35]:
for relationship in graph_doc[0].relationships:
  print(relationship)

source=Node(id='Mr.', type='Person', properties={}) target=Node(id='Pancreatic Cancer', type='Disease', properties={}) type='HAS_DISEASE' properties={}
source=Node(id='Mr.', type='Person', properties={}) target=Node(id='Ulcerative Colitis', type='Disease', properties={}) type='HAS_DISEASE' properties={}
source=Node(id='Mr.', type='Person', properties={}) target=Node(id='Hypertension', type='Disease', properties={}) type='HAS_DISEASE' properties={}
source=Node(id='Mr.', type='Person', properties={}) target=Node(id='Endoscopic Retrograde Cholangiopancreatography', type='Medical procedure', properties={}) type='UNDERWENT_PROCEDURE' properties={}
source=Node(id='Mr.', type='Person', properties={}) target=Node(id='Total Abdominal Colectomy', type='Medical procedure', properties={}) type='UNDERWENT_PROCEDURE' properties={}
source=Node(id='Total Abdominal Colectomy', type='Medical procedure', properties={}) target=Node(id='End-Ileostomy', type='Medical procedure', properties={}) type='HAS_PRO

In [36]:
graph.add_graph_documents(graph_doc)

In [40]:
from langchain.chains import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
    google_api_key="",
    temperature=0,
    allow_dangerous_requests=True,
)

In [ ]:
text="woman with diabetes, hypertension, hyperlipidemia and obesity, with a one to two months of chest burning with exertion. for the past six months, she has been participating in a new vigorous exercise program to lose weight. her symptoms do gradually resolve with rest, but they have started to occur now with walking. she does acknowledge that there is associated nausea, diaphoresis and shortness of breath. now recently she started to get symptoms for the past two days while at rest. she was referred for an outpatient exercise tolerance test, where she had chest pain and significant ekg changes. she was referred to for cardiac catheterization today, which revealed significant left main artery disease. just prior to her transfer to , she did complain of chest pain and back pain at about 7/10 intensity and a nitroglycerin drip was started and she received 5 mg of iv lopressor and 2 mg of iv ativan, which did resolve her pain. while she was in the cath lab, she had an intraaortic balloon pump placed, and now she is not actively complaining of any pain. review of systems: she denies any orthopnea, lower extremity edema, but she does acknowledge that she does have dysmenorrhea. prior medical history: she has diabetes mellitus, type 2, diagnosed in . she has been able to control it with diet since ; hypertension and obesity; however, she has lost 40 pounds in the last six months. she has an allergy to shellfish. medications on transfers to : atenolol 50 mg a day, lisinopril 10 mg a day, lipitor 20 mg a day, aspirin 325 mg once a day, progesterone, nitroglycerin drip, lopressor, ativan"

In [43]:
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm_query, verbose=True,allow_dangerous_requests=True)
response = chain.invoke(text)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease {name: "Pancreatic Cancer"})-[r:HAS_FINDING]->(mf:Medical_finding)
RETURN d,r,mf

Full Context:
[]

> Finished chain.


In [ ]:
response

{'query': 'woman with diabetes, hypertension, hyperlipidemia and obesity, with a one to two months of chest burning with exertion. for the past six months, she has been participating in a new vigorous exercise program to lose weight. her symptoms do gradually resolve with rest, but they have started to occur now with walking. she does acknowledge that there is associated nausea, diaphoresis and shortness of breath. now recently she started to get symptoms for the past two days while at rest. she was referred for an outpatient exercise tolerance test, where she had chest pain and significant ekg changes. she was referred to for cardiac catheterization today, which revealed significant left main artery disease. just prior to her transfer to , she did complain of chest pain and back pain at about 7/10 intensity and a nitroglycerin drip was started and she received 5 mg of iv lopressor and 2 mg of iv ativan, which did resolve her pain. while she was in the cath lab, she had an intraaortic 

In [ ]:
response = chain.invoke("What are the possible complications of peripheral vascular disease")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Disease {id: "peripheral vascular disease"})<-[:HOSPITAL_COMPLICATION]-(patient:Patient)
RETURN DISTINCT p, patient

Full Context:
[]

> Finished chain.
{'query': 'What are the possible complications of peripheral vascular disease', 'result': "I don't know the answer."}


In [ ]:
!pip install langchain_neo4j

In [ ]:
graph_qa = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    cypher_template="""
    MATCH (d:Disease {id: $disease})-[:LIKELY]->(complication:Disease)
    RETURN complication.id
    """,
    return_intermediate_steps=True,
    allow_dangerous_requests=True,
)
response = graph_qa.invoke("What are the possible complications of peripheral vascular disease in this case")
response

{'query': 'What are the possible complications of peripheral vascular disease in this case',
 'result': "I don't know the answer.",
 'intermediate_steps': [{'query': 'MATCH (d:Document)-[:HAS_MAJOR_TOPIC]->(:MajorTopic {name: "peripheral vascular disease"})\nRETURN d.title, d.abstract'},
  {'context': []}]}

In [ ]:
graph_qa = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    cypher_template="""
    MATCH (d:Disease {id: $disease})-[r]->(complication:Disease)
    RETURN type(r) AS relationship_type, complication.id
    """,
    return_intermediate_steps=True,
    allow_dangerous_requests=True,
    verbose=True  # Enable verbose mode if available
)



In [ ]:
response = graph_qa.invoke("What are the possible complications of peripheral vascular disease")
print(response)



> Entering new GraphCypherQAChain chain...


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API Key not found. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API Key not found. Please pass a valid API key."
]